In [3]:
import textwrap
from typing import Optional, List

import pydantic
from langchain import LLMChain, OpenAI
from langchain.agents import AgentExecutor, Tool
from langchain.agents.chat.base import ChatAgent
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.prompts import BaseChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage, OutputParserException, BaseOutputParser
from langchain.tools import BaseTool
from langchain.output_parsers import RetryWithErrorOutputParser

from pydantic import Field, BaseModel, validator

In [1]:
from qabot import ask_wikidata, ask_file

In [2]:
print(ask_file("How many men were aboard the splender?", 'data/titanic_.csv'))



> Entering new AgentExecutor chain...
{
    "action": "Data Op",
    "action_input": "How many men were aboard the Splender? Please provide the relevant table and column names."
}

> Entering new AgentExecutor chain...
format error 1 validation error for CustomLLMResponse
__root__
  Expecting value: line 1 column 1 (char 0) (type=value_error.jsondecode; msg=Expecting value; doc=Query Inspector:
```
SELECT COUNT(*) FROM titanic_ WHERE Sex = 'male' AND Name LIKE '%Splender%';
```; pos=0; lineno=1; colno=1)
{
"type": "answer",
"rational": "To find the number of male passengers with the name Splender, we need to filter the table by Sex and Name columns. We can do this by using the WHERE clause with appropriate conditions. Then we can count the number of rows returned using the COUNT() function. ",
"input": "",
"result": {
    "output": "There is 1 male passenger with the name Splender.",
    "query": "SELECT COUNT(*) FROM titanic_ WHERE Sex = 'male' AND Name LIKE '%Splender%';"
}
}




>

In [5]:
from qabot.tools.duckdb_execute_tool import DuckDBTool
from qabot.duckdb_query import run_sql_catch_error
from qabot.tools.describe_duckdb_table import describe_table_or_view

In [6]:
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[BaseTool]

    def format_messages(self, **kwargs):
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        if len(intermediate_steps) == 0:
            thoughts = "Thought: I should execute a query before giving my answer\n"
        else:
            thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            

        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f'"{tool.name}": {tool.description}' for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])

        kwargs['output_instructions'] = output_instructions

        # TODO probably good to get the updated table names here
        # table_names =
        formatted = self.template.format(**kwargs)
        return [
            HumanMessage(content=formatted)
        ]

In [7]:
tools = [
        Tool(
            name="Show Tables",
            func=lambda _: run_sql_catch_error(database, "show tables;"),
            description="Useful to show the available tables and views. Empty input required."
        ),
        Tool(
            name="Describe Table",
            func=lambda table: describe_table_or_view(database, table),
            description="Useful to show the column names and types of a table or view. Also shows the first few rows. Use a valid table name as the input."
        ),
        Tool(
            name="Query Inspector",
            func=lambda query: query.strip('"').strip("'"),
            description="Useful to show the query before execution. Always inspect your query before execution. Input MUST be on one line."
        ),
        DuckDBTool(engine=database),
    ]

NameError: name 'database' is not defined

In [4]:
prompt = CustomPromptTemplate(
        template=template,
        tools=tools,
        # This omits the `agent_scratchpad`, `tools`, `tool_names` variables because
        # they are generated dynamically by the CustomPromptTemplate.
        input_variables=["input", "intermediate_steps", "table_names"]
    )

NameError: name 'CustomPromptTemplate' is not defined